In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
from tqdm import trange
import numpy as np
import random
import json
import os
import tensorflow_datasets


mnist = tensorflow_datasets.load('mnist')


np.random.seed(0)
NUM_USERS = 100
NUM_LABELS = 2

# Setup directory for train/test data
train_path = './data/train/fashion_train.json'
test_path = './data/test/fashion_test.json'
dir_path = os.path.dirname(train_path)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
dir_path = os.path.dirname(test_path)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
import tensorflow as tf
# Import Fashion MNIST
fashion_data = fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

print("hello")
#fashion_full = list(zip(fashion_data.train, fashion_data.test))
print(type(fashion_data))
mnist_data_image = []
mnist_data_lable = []
mnist_data_image.extend(train_images)
mnist_data_image.extend(test_images)
mnist_data_lable.extend(train_labels)
mnist_data_lable.extend(test_labels)

mu = np.mean(mnist_data_image)
sigma = np.std(mnist_data_image)
nom_Fashion_data = (mnist_data_image - mu)/(sigma+0.001)
nom_Fashion_lable = np.array(mnist_data_lable)
mnist_data = []
for i in trange(10):
    idx = nom_Fashion_lable == i
    mnist_data.append(nom_Fashion_data[idx])

print([len(v) for v in mnist_data])


###### CREATE USER DATA SPLIT #######
# Assign 100 samples to each user
X = [[] for _ in range(NUM_USERS)]
y = [[] for _ in range(NUM_USERS)]
idx = np.zeros(10, dtype=np.int64)
for user in range(NUM_USERS):
    for j in range(NUM_LABELS):  # 3 labels for each users
        # l = (2*user+j)%10
        l = (user + j) % 10
        print("L:", l)
        X[user] += mnist_data[l][idx[l]:idx[l]+10].tolist()
        y[user] += (l*np.ones(10)).tolist()
        idx[l] += 10

print("IDX1:", idx)  # counting samples for each labels

# Assign remaining sample by power law
user = 0
props = np.random.lognormal(
    0, 2., (10, NUM_USERS, NUM_LABELS))  # last 5 is 5 labels
# print("here:",props/np.sum(props,(1,2), keepdims=True))
props = np.array([[[len(v)-100]] for v in mnist_data]) * \
    props/np.sum(props, (1, 2), keepdims=True)
#idx = 1000*np.ones(10, dtype=np.int64)
# print("here2:",props)
for user in trange(NUM_USERS):
    for j in range(NUM_LABELS):  # 4 labels for each users
        # l = (2*user+j)%10
        l = (user + j) % 10
        #num_samples = int(props[l,user//int(NUM_USERS/10),j]) *10
        num_samples = int(props[l, user, j]) * NUM_USERS * 2
        #num_samples = min(num_samples,200)
        # print(num_samples)
        if idx[l] + num_samples < len(mnist_data[l]):
            X[user] += mnist_data[l][idx[l]:idx[l]+num_samples].tolist()
            y[user] += (l*np.ones(num_samples)).tolist()
            idx[l] += num_samples

print("IDX2:", idx)  # counting samples for each labels
# Create data structure
train_data = {'users': [], 'user_data': {}, 'num_samples': []}
test_data = {'users': [], 'user_data': {}, 'num_samples': []}

# Setup 5 users
# for i in trange(5, ncols=120):
for i in range(NUM_USERS):
    uname = 'f_{0:05d}'.format(i)

    combined = list(zip(X[i], y[i]))
    random.shuffle(combined)
    X[i][:], y[i][:] = zip(*combined)
    num_samples = len(X[i])
    train_len = int(0.75*num_samples)
    test_len = num_samples - train_len

    train_data['users'].append(uname)
    train_data['user_data'][uname] = {
        'x': X[i][:train_len], 'y': y[i][:train_len]}
    train_data['num_samples'].append(train_len)
    test_data['users'].append(uname)
    test_data['user_data'][uname] = {
        'x': X[i][train_len:], 'y': y[i][train_len:]}
    test_data['num_samples'].append(test_len)

print("Num_samples:", train_data['num_samples'])
print("Total_samples:", sum(train_data['num_samples']))

with open(train_path, 'w') as outfile:
    json.dump(train_data, outfile)
with open(test_path, 'w') as outfile:
    json.dump(test_data, outfile)

print("Finish Generating Samples")

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
4423680/4422102 [==============================] - 0s 0us/step
hello
<class 'module'>


100%|██████████| 10/10 [00:00<00:00, 86.43it/s]


[7000, 7000, 7000, 7000, 7000, 7000, 7000, 7000, 7000, 7000]
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4

  4%|▍         | 4/100 [00:00<00:13,  7.35it/s]


L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
L: 0
L: 1
L: 1
L: 2
L: 2
L: 3
L: 3
L: 4
L: 4
L: 5
L: 5
L: 6
L: 6
L: 7
L: 7
L: 8
L: 8
L: 9
L: 9
L: 0
IDX1: [200 200 200 200 200 200 200 200 200 200]


100%|██████████| 100/100 [00:05<00:00, 17.62it/s]


IDX2: [6600 6600 6800 6000 6600 6800 6600 6400 6800 6400]
Num_samples: [615, 1965, 315, 1365, 915, 1815, 1365, 465, 1365, 15, 465, 1665, 765, 1515, 165, 15, 2265, 615, 615, 915, 465, 915, 465, 165, 15, 1515, 15, 765, 2415, 615, 465, 315, 465, 2115, 1365, 15, 315, 615, 315, 765, 15, 465, 15, 165, 1665, 15, 15, 3015, 165, 1515, 15, 915, 465, 15, 1665, 165, 15, 15, 15, 15, 1665, 15, 165, 465, 1365, 165, 15, 315, 165, 15, 15, 165, 15, 15, 15, 15, 15, 15, 15, 165, 165, 15, 15, 15, 15, 15, 15, 15, 15, 165, 15, 15, 15, 15, 165, 15, 15, 15, 15, 15]
Total_samples: 49200
Finish Generating Samples


In [ ]:
! pip install Flearn 
import h5py
import matplotlib.pyplot as plt
import numpy as np
import argparse
import importlib
import random
import os
import tensorflow.compat.v1 as tf
import Flearn
tf.disable_v2_behavior() 
from flearn.utils.plot_utils import *
from flearn.utils.model_utils import read_data
import matplotlib
matplotlib.use('Agg')

# GLOBAL PARAMETERS
OPTIMIZERS = ['fedsgd', 'fedfedl']

DATASETS = ['nist', 'mnist', 'fashion_mnist']  # NIST is EMNIST in the paper

MODEL_PARAMS = {
    'sent140.bag_dnn': (2,),  # num_classes
    'sent140.stacked_lstm': (25, 2, 100),  # seq_len, num_classes, num_hidden
    # seq_len, num_classes, num_hidden
    'sent140.stacked_lstm_no_embeddings': (25, 2, 100),
    # num_classes, should be changed to 62 when using EMNIST
    'nist.mclr': (62,),
    'nist.cnn': (62,),
    'mnist.mclr': (10,),  # num_classes
    'mnist.cnn': (10,),  # num_classes
    'fashion_mnist.mclr': (10,),
    'fashion_mnist.cnn': (10,),
    'shakespeare.stacked_lstm': (80, 80, 256),  # seq_len, emb_dim, num_hidden
    'synthetic.mclr': (10, )  # num_classes
}


def read_options(num_users=5, loc_ep=10, Numb_Glob_Iters=100, lamb=0, learning_rate=0.01, hyper_learning_rate = 0.01, alg='fedprox', weight=True, batch_size=0, times = 10, rho = 0, dataset="mnist"):
    ''' Parse command line arguments or load defaults '''
    parser = argparse.ArgumentParser()

    parser.add_argument('--optimizer',
                        help='name of optimizer;',
                        type=str,
                        choices=OPTIMIZERS,
                        default=alg)  # fedavg, fedprox
    parser.add_argument('--dataset',
                        help='name of dataset;',
                        type=str,
                        choices=DATASETS,
                        default=dataset)
    parser.add_argument('--model',
                        help='name of model;',
                        type=str,
                        default='mclr.py')  # 'stacked_lstm.py'
    parser.add_argument('--num_rounds',
                        help='number of rounds to simulate;',
                        type=int,
                        default=Numb_Glob_Iters)
    parser.add_argument('--eval_every',
                        help='evaluate every __ rounds;',
                        type=int,
                        default=1)
    parser.add_argument('--clients_per_round',
                        help='number of clients trained per round;',
                        type=int,
                        default=num_users)
    parser.add_argument('--batch_size',
                        help='batch size when clients train on data;',
                        type=int,
                        default=batch_size
                        )  # 0 is full dataset
    parser.add_argument('--num_epochs',
                        help='number of epochs when clients train on data;',
                        type=int,
                        default=loc_ep)
    parser.add_argument('--learning_rate',
                        help='learning rate for inner solver;',
                        type=float,
                        default=learning_rate)  # 0.003
    parser.add_argument('--hyper_learning_rate',
                        help='learning rate for inner solver;',
                        type=float,
                        default=hyper_learning_rate)  # 0.003
    parser.add_argument('--mu',
                        help='constant for prox;',
                        type=float,
                        default=0.)  # 0.01
    parser.add_argument('--seed',
                        help='seed for randomness;',
                        type=int,
                        default=0)
    parser.add_argument('--weight',
                        help='enable weight value;',
                        type=int,
                        default=weight)
    parser.add_argument('--lamb',
                        help='Penalty value for proximal term;',
                        type=int,
                        default=lamb)
    parser.add_argument('--times',
                        help='Number of running time;',
                        type=int,
                        default=1)
    parser.add_argument('--rho',
                        help='Condition number only for synthetic data;',
                        type=float,
                        default=rho)
            
    try:
        parsed = vars(parser.parse_args())
    except IOError as msg:
        parser.error(str(msg))

    # Set seeds
    random.seed(1 + parsed['seed'])
    np.random.seed(12 + parsed['seed'])
    tf.set_random_seed(123 + parsed['seed'])

    # load selected model
    # all synthetic datasets use the same model
    if parsed['dataset'].startswith("synthetic"):
        model_path = '%s.%s.%s.%s' % (
            'flearn', 'models', 'synthetic', parsed['model'])
    else:
        model_path = '%s.%s.%s.%s' % (
            'flearn', 'models', parsed['dataset'], parsed['model'])

    # mod = importlib.import_module(model_path)
    import flearn.models.mnist.mclr as mclr
    mod = mclr
    learner = getattr(mod, 'Model')

    # load selected trainer
    opt_path = 'flearn.trainers.%s' % parsed['optimizer']
    mod = importlib.import_module(opt_path)
    optimizer = getattr(mod, 'Server')

    # add selected model parameter
    parsed['model_params'] = MODEL_PARAMS['.'.join(
        model_path.split('.')[2:-1])]
    # parsed['model_params'] = MODEL_PARAMS['mnist.mclr']

    # print and return
    maxLen = max([len(ii) for ii in parsed.keys()])
    fmtString = '\t%' + str(maxLen) + 's : %s'
    print('Arguments:')
    for keyPair in sorted(parsed.items()):
        print(fmtString % keyPair)

    return parsed, learner, optimizer


def main(num_users=5, loc_ep=10, Numb_Glob_Iters=100, lamb=0, learning_rate=0.01,hyper_learning_rate= 0.01, alg='fedprox', weight=True, batch_size=0, times =10, rho = 0, dataset="mnist"):
    # suppress tf warnings
    tf.logging.set_verbosity(tf.logging.WARN)

    # parse command line arguments
    options, learner, optimizer = read_options(
        num_users, loc_ep, Numb_Glob_Iters, lamb, learning_rate,hyper_learning_rate, alg, weight, batch_size, times, rho, dataset)

    # read data
    train_path = os.path.join('data', options['dataset'], 'data', 'train')
    test_path = os.path.join('data', options['dataset'], 'data', 'test')
    dataset = read_data(train_path, test_path)

       # call appropriate trainer
    for i in range(times):
        # Set seeds
        random.seed(1 + i)
        np.random.seed(12 + i)
        tf.set_random_seed(123 + i)
        print('......time for runing......',i)
        t = optimizer(options, learner, dataset)
        t.train(i)
    average_data(num_users=num_users, loc_ep1=loc_ep, Numb_Glob_Iters=Numb_Glob_Iters, lamb=lamb, learning_rate=learning_rate, hyper_learning_rate = hyper_learning_rate, algorithms=alg, batch_size=batch_size, dataset=dataset, rho = rho, times = times)



# if _name_ == '_main_':
#     algorithms_list = ["fedfedl", "fedsgd",
#                        "fedfedl", "fedsgd", 
#                        "fedfedl", "fedsgd"]
#     lamb_value = [0, 0, 0, 0, 0, 0]
#     learning_rate = [0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
#     local_ep = [10,10, 20, 20, 50, 50]
#     batch_size = [20,20, 20, 20, 20, 20]
#     DATA_SET = "nist"
#     number_users = 30

#     for i in range(len(algorithms_list)):
#         main(num_users=number_users, loc_ep=local_ep[i], Numb_Glob_Iters=1000, lamb=lamb_value[i],
#              learning_rate=learning_rate[i], alg=algorithms_list[i], batch_size=batch_size[i], rho = rho[i], dataset=DATA_SET)

#     plot_summary_three_figures(num_users=number_users, loc_ep1=local_ep, Numb_Glob_Iters=800, lamb=lamb_value,
#                             learning_rate=learning_rate, algorithms_list=algorithms_list, batch_size=batch_size, rho = rho[i], dataset=DATA_SET)
#     print("-- FINISH -- :",)


if _name_ == '_main_':
    
    algorithms_list =  ["fedfedl","fedsgd","fedfedl", "fedfedl","fedsgd","fedfedl", "fedfedl","fedsgd","fedfedl"]
    rho = [0,0,0,0,0,0,0,0,0,0,0,0]
    lamb_value = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    learning_rate = [0.003, 0.003, 0.015, 0.003, 0.003, 0.015, 0.003, 0.003, 0.015]
    hyper_learning_rate = [0.2, 0, 0.5, 0.2, 0, 0.5, 0.2, 0, 0.5]
    local_ep = [10, 10, 10, 20, 20, 20, 40, 40, 40]
    batch_size = [20, 20, 0, 20, 20, 0, 20, 20, 0]
    DATA_SET = "nist"
    number_users = 10
    
    # for i in range(len(algorithms_list)):
    #     main(num_users=number_users, loc_ep=local_ep[i], Numb_Glob_Iters=800, lamb=lamb_value[i],
    #          learning_rate=learning_rate[i],hyper_learning_rate=hyper_learning_rate[i],  alg=algorithms_list[i], batch_size=batch_size[i], rho = rho[i], dataset=DATA_SET)

    plot_summary_nist(num_users=number_users, loc_ep1=local_ep, Numb_Glob_Iters=800, lamb=lamb_value,
                               learning_rate=learning_rate, hyper_learning_rate = hyper_learning_rate, algorithms_list=algorithms_list, batch_size=batch_size, rho = rho, dataset=DATA_SET)
    print("-- FINISH -- :",)

TabError: ignored